In [98]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
import json
import random
import tensorflow as tf   
tf.get_logger().setLevel('ERROR')

In [99]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [91]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [92]:
def predict_class(sentence, model , words , classes):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.00
    results = [[i,r] for i,r in enumerate(res) if r>=ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    print(return_list)
    return return_list

In [93]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [94]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [95]:
def call_model(sentence , qtype):
    no = qtype
    model = load_model('models/model' + no + '.h5')
    words = pickle.load(open('type' + no + '/words.pkl','rb'))
    classes = pickle.load(open('type' + no + '/classes.pkl','rb'))
    found_intent = predict_class(sentence , model , words , classes)[0]['intent']
    return print("Intent is " , found_intent)

In [96]:
def main(question):
    print("Finding the type")
    model = load_model('models/model.h5')
    words = pickle.load(open('main_model/words.pkl','rb'))
    classes = pickle.load(open('main_model/classes.pkl','rb'))
    qtype = predict_class(question , model , words , classes)[0]['intent']
    print("\nQuestion type is" , qtype , end='\n\n')
    if qtype == 'type1':
        call_model(question , '1')
    elif qtype == 'type2':
        call_model(question , '2')
    elif qtype == 'type3':
        call_model(question , '3')

In [97]:
main("Hello everyone")


Finding the type


 [{'intent': 'type1', 'probability': '0.9999417'}, {'intent': 'type2', 'probability': '3.1881755e-05'}, {'intent': 'type3', 'probability': '2.648187e-05'}]


Question type is type1


 [{'intent': 'greeting', 'probability': '0.99703'}, {'intent': 'info_iitmandi', 'probability': '0.0005976363'}, {'intent': 'contact_information', 'probability': '0.0005308608'}, {'intent': 'alumni', 'probability': '0.00041999473'}, {'intent': 'creation', 'probability': '0.0003155136'}, {'intent': 'placements', 'probability': '0.00026389695'}, {'intent': 'options', 'probability': '0.00024985807'}, {'intent': 'medical_facilities', 'probability': '0.00023290576'}, {'intent': 'thanks', 'probability': '0.00020407373'}, {'intent': 'goodbye', 'probability': '0.00015524711'}]


 Intent is  greeting
